In [2]:
import torch

In [3]:
torch.cpu.is_available()

True

In [4]:
parity_check_matrix = [
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
    [0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1]
]
parity_check_matrix_colsize = len(parity_check_matrix[0])
parity_check_matrix_rowsize = len(parity_check_matrix)
# first hidden layer node number count 1 number in parity_matrix
hidden_layer_node = sum(sum(row) for row in parity_check_matrix)


print("parity_check_matrix_rowsize : ",parity_check_matrix_rowsize)
print("parity_check_matrix_colsize : ",parity_check_matrix_colsize)
print("hidden_layer_node : ",hidden_layer_node)

parity_check_matrix_rowsize :  4
parity_check_matrix_colsize :  15
hidden_layer_node :  33


In [5]:
CN2VN_pos = []
max_row_degree =0 
for r in range(0,len(parity_check_matrix)):
    tmp=[]
    for c in range(0,len(parity_check_matrix[0])):
        if parity_check_matrix[r][c]==1:
            tmp.append(c)
    max_row_degree = max(max_row_degree,len(tmp))
    CN2VN_pos.append(tmp)
# print("CN2VN_pos:")
# CN2VN_pos
print("max_row_degree : ",max_row_degree)

max_row_degree :  9


In [6]:
VN2CN=[]
max_col_degree=0
for c in range(0,len(parity_check_matrix[0])):
    tmp =[]
    for r in range(0,len(parity_check_matrix)):
        if parity_check_matrix[r][c]==1:
            tmp.append(r)
    VN2CN.append(tmp)
    max_col_degree = max(max_col_degree,len(tmp))
# print("VN2CN : ")
# VN2CN
print("max_col_degree : ",max_col_degree)

max_col_degree :  4


### Write parity_check_matrix to txt file 

In [7]:
with open("BCH_15_11.txt", "w") as txt_file:
    txt_file.write("{} {} \n".format(parity_check_matrix_colsize ,parity_check_matrix_rowsize))
    txt_file.write("{} {} \n".format(max_col_degree,max_row_degree))
    for i in range(0,len(VN2CN)):
        txt_file.write("{} ".format(len(VN2CN[i])))
    txt_file.write("\n")
    for i in range(0,len(CN2VN_pos)):
        txt_file.write("{} ".format(len(CN2VN_pos[i])))
    txt_file.write("\n")
    for i in range(0,len(VN2CN)):
        for j in range(0,len(VN2CN[i])):
            txt_file.write("{} ".format(VN2CN[i][j]+1))
        if len(VN2CN[i])!=max_col_degree:
            for j in range(0,max_col_degree-len(VN2CN[i])):
                txt_file.write("{} ".format(0))
        txt_file.write("\n")
    for i in range(0,len(CN2VN_pos)):
        for j in range(0,len(CN2VN_pos[i])):
            txt_file.write("{} ".format(CN2VN_pos[i][j]+1))
        if len(CN2VN_pos[i])!=max_row_degree:
            for j in range(0,max_row_degree-len(CN2VN_pos[i])):
                txt_file.write("{} ".format(0))
        txt_file.write("\n")
    txt_file.write("\n")
        

In [8]:
from collections import defaultdict
connect_first_layer = []
dict_update_cn = {}
dict_update_cn_arr = defaultdict(list)
dict_vn_pos = defaultdict(list)
# for first layer mask
for CN in range(0,len(CN2VN_pos)):
    for c in range(0,len(CN2VN_pos[CN])):
        tmp = CN2VN_pos[CN][0:c] + CN2VN_pos[CN][c+1:len(CN2VN_pos[CN])]
        connect_first_layer.append(tmp)
        dict_update_cn[tuple(tmp)]=(CN2VN_pos[CN][c],CN) # [connect to VNs] - update(CN->CN2VN_pos[r][c]) (VN,CN)
        dict_update_cn_arr[(CN2VN_pos[CN][c],CN)].append(tuple(tmp))
        # print(dict_update_cn[tuple(tmp)], "-",tuple(tmp))
        

# for second layer mask
# save neural network node position ([connect to VNs]- node_position)
# print("second layer : ")
dict_connect_first_layer ={}
for idx,arr in enumerate(connect_first_layer):
    dict_connect_first_layer[tuple(arr)]=idx 
    # print(idx," - ",tuple(arr))

for key in sorted(dict_update_cn_arr.keys(),key=lambda x:x[1]):
    VN = key[0]
    for arr in dict_update_cn_arr[key]:
        dict_vn_pos[VN].append(dict_connect_first_layer[arr])
# sort 
sorted_keys = sorted(dict_vn_pos.keys(),key=lambda x:x)
dict_vn_pos_sort = {key: dict_vn_pos[key] for key in sorted_keys}
dict_vn_pos=dict_vn_pos_sort.copy()
# show update key with node
for key in dict_vn_pos.keys():
    print(key,"-",dict_vn_pos[key])

#constuct third mask of VN->CN update (record VN->CN node postion in neural network)
dict_vn_to_cn_node = defaultdict(list)
idx=0
for VN in dict_vn_pos.keys():
    for CN in VN2CN[VN]:
        dict_vn_to_cn_node[(VN,CN)] = idx
        idx+=1
for key in dict_vn_to_cn_node.keys():
    print(key,"-",dict_vn_to_cn_node[key])

# construct fourth mask of  CN ->VN + channel LLR = output
dict_cn_to_output = defaultdict(list)
idx=0
for CN in range(0,len(CN2VN_pos)):
    for VN in CN2VN_pos[CN]:
        dict_cn_to_output[VN].append(idx)
        idx+=1
# sort 
sorted_keys = sorted(dict_cn_to_output.keys(),key=lambda x:x)
dict_cn_to_output_sort = {key: dict_cn_to_output[key] for key in sorted_keys}
dict_cn_to_output=dict_cn_to_output_sort.copy()
dict_cn_to_output

0 - [0]
1 - [9]
2 - [17]
3 - [25]
4 - [1, 18, 26]
5 - [10, 19, 27]
6 - [28]
7 - [2, 11]
8 - [12, 20]
9 - [3, 21, 29]
10 - [4, 30]
11 - [5, 13, 31]
12 - [6, 14, 22]
13 - [7, 15, 23]
14 - [8, 16, 24, 32]
(0, 0) - 0
(1, 1) - 1
(2, 2) - 2
(3, 3) - 3
(4, 0) - 4
(4, 2) - 5
(4, 3) - 6
(5, 1) - 7
(5, 2) - 8
(5, 3) - 9
(6, 3) - 10
(7, 0) - 11
(7, 1) - 12
(8, 1) - 13
(8, 2) - 14
(9, 0) - 15
(9, 2) - 16
(9, 3) - 17
(10, 0) - 18
(10, 3) - 19
(11, 0) - 20
(11, 1) - 21
(11, 3) - 22
(12, 0) - 23
(12, 1) - 24
(12, 2) - 25
(13, 0) - 26
(13, 1) - 27
(13, 2) - 28
(14, 0) - 29
(14, 1) - 30
(14, 2) - 31
(14, 3) - 32


{0: [0],
 1: [9],
 2: [17],
 3: [25],
 4: [1, 18, 26],
 5: [10, 19, 27],
 6: [28],
 7: [2, 11],
 8: [12, 20],
 9: [3, 21, 29],
 10: [4, 30],
 11: [5, 13, 31],
 12: [6, 14, 22],
 13: [7, 15, 23],
 14: [8, 16, 24, 32]}

### Construct every layer mask

In [9]:
# construct first layer mask (CN update)
first_layer_mask = [[False]*parity_check_matrix_colsize for i in range(0,hidden_layer_node)]

print("layer_mask rowsize is :",len(first_layer_mask)," , colsize : ",len(first_layer_mask[0]))

for i in range(0,len(connect_first_layer)):
    for num in connect_first_layer[i]:
        first_layer_mask[i][num]=True

layer_mask rowsize is : 33  , colsize :  15


In [10]:
# construct CN2VN mask - second layer connect table matrix(mask)
CN2VN_mask_VNUpdate = [[False]*hidden_layer_node for i in range(0,hidden_layer_node)]
idx=0
tmp=[]
for VN in dict_vn_pos.keys():
    if(len(dict_vn_pos[VN])==1):
        idx+=1
        tmp.append([VN2CN[VN][0],VN])
        continue
    for i,node1 in enumerate(dict_vn_pos[VN]):
        for node2 in dict_vn_pos[VN]:
            if node1!=node2:
                CN2VN_mask_VNUpdate[idx][node2]=True
        idx+=1
        tmp.append([VN2CN[VN][i],VN]) # CN,VN
true_indices_per_row = [[idx for idx, val in enumerate(row) if val] for row in CN2VN_mask_VNUpdate]
for idx,arr in enumerate(true_indices_per_row):
    print("neourn : {} - {} | VN {} -> CN {}".format(idx, arr, tmp[idx][1], tmp[idx][0]))

neourn : 0 - [] | VN 0 -> CN 0
neourn : 1 - [] | VN 1 -> CN 1
neourn : 2 - [] | VN 2 -> CN 2
neourn : 3 - [] | VN 3 -> CN 3
neourn : 4 - [18, 26] | VN 4 -> CN 0
neourn : 5 - [1, 26] | VN 4 -> CN 2
neourn : 6 - [1, 18] | VN 4 -> CN 3
neourn : 7 - [19, 27] | VN 5 -> CN 1
neourn : 8 - [10, 27] | VN 5 -> CN 2
neourn : 9 - [10, 19] | VN 5 -> CN 3
neourn : 10 - [] | VN 6 -> CN 3
neourn : 11 - [11] | VN 7 -> CN 0
neourn : 12 - [2] | VN 7 -> CN 1
neourn : 13 - [20] | VN 8 -> CN 1
neourn : 14 - [12] | VN 8 -> CN 2
neourn : 15 - [21, 29] | VN 9 -> CN 0
neourn : 16 - [3, 29] | VN 9 -> CN 2
neourn : 17 - [3, 21] | VN 9 -> CN 3
neourn : 18 - [30] | VN 10 -> CN 0
neourn : 19 - [4] | VN 10 -> CN 3
neourn : 20 - [13, 31] | VN 11 -> CN 0
neourn : 21 - [5, 31] | VN 11 -> CN 1
neourn : 22 - [5, 13] | VN 11 -> CN 3
neourn : 23 - [14, 22] | VN 12 -> CN 0
neourn : 24 - [6, 22] | VN 12 -> CN 1
neourn : 25 - [6, 14] | VN 12 -> CN 2
neourn : 26 - [15, 23] | VN 13 -> CN 0
neourn : 27 - [7, 23] | VN 13 -> CN 1
n

In [11]:
#construct third mask(VN->CN)
VN2CN_mask_CNUpdate = [[False]*hidden_layer_node for i in range(0,hidden_layer_node)]
# 照著cn0->vn1 -> cn0->vn2 ...順序
idx=0
CN_2_VN_record=[]
for CN in range(0,parity_check_matrix_rowsize):
    #  CN->VN1 update
    for VN1 in  CN2VN_pos[CN]: 
        for VN2 in CN2VN_pos[CN]:
            if VN1!=VN2:
                # print(dict_vn_to_cn_node[(VN2,CN)])
                VN2CN_mask_CNUpdate[idx][dict_vn_to_cn_node[(VN2,CN)]]=True
        idx+=1
        CN_2_VN_record.append([CN,VN1])
print("idx : ",idx)
true_indices_per_row = [[idx for idx, val in enumerate(row) if val] for row in VN2CN_mask_CNUpdate]
for idx,arr in enumerate(true_indices_per_row):
    # print("node : ",idx," - ",arr,"| CN",CN_2_VN_record[idx][0],"->","VN",CN_2_VN_record[idx][1])
    print("neourn : {} - {} | CN {} -> VN {}".format(idx, arr, CN_2_VN_record[idx][0], CN_2_VN_record[idx][1]))
# print(VN2CN_mask_CNUpdate)

idx :  33
neourn : 0 - [4, 11, 15, 18, 20, 23, 26, 29] | CN 0 -> VN 0
neourn : 1 - [0, 11, 15, 18, 20, 23, 26, 29] | CN 0 -> VN 4
neourn : 2 - [0, 4, 15, 18, 20, 23, 26, 29] | CN 0 -> VN 7
neourn : 3 - [0, 4, 11, 18, 20, 23, 26, 29] | CN 0 -> VN 9
neourn : 4 - [0, 4, 11, 15, 20, 23, 26, 29] | CN 0 -> VN 10
neourn : 5 - [0, 4, 11, 15, 18, 23, 26, 29] | CN 0 -> VN 11
neourn : 6 - [0, 4, 11, 15, 18, 20, 26, 29] | CN 0 -> VN 12
neourn : 7 - [0, 4, 11, 15, 18, 20, 23, 29] | CN 0 -> VN 13
neourn : 8 - [0, 4, 11, 15, 18, 20, 23, 26] | CN 0 -> VN 14
neourn : 9 - [7, 12, 13, 21, 24, 27, 30] | CN 1 -> VN 1
neourn : 10 - [1, 12, 13, 21, 24, 27, 30] | CN 1 -> VN 5
neourn : 11 - [1, 7, 13, 21, 24, 27, 30] | CN 1 -> VN 7
neourn : 12 - [1, 7, 12, 21, 24, 27, 30] | CN 1 -> VN 8
neourn : 13 - [1, 7, 12, 13, 24, 27, 30] | CN 1 -> VN 11
neourn : 14 - [1, 7, 12, 13, 21, 27, 30] | CN 1 -> VN 12
neourn : 15 - [1, 7, 12, 13, 21, 24, 30] | CN 1 -> VN 13
neourn : 16 - [1, 7, 12, 13, 21, 24, 27] | CN 1 -> VN 14

In [12]:
# construct fouth final CN update to result
CN2VN_mask_output = [[False]*hidden_layer_node for i in range(0,parity_check_matrix_colsize)]
for VN in range(0,len(dict_cn_to_output)):
    for node in dict_cn_to_output[VN]:
        CN2VN_mask_output[VN][node] = True
    # print(CN2VN_mask_output[VN])
    
true_indices_per_row = [[idx for idx, val in enumerate(row) if val] for row in CN2VN_mask_output]
for idx,arr in enumerate(true_indices_per_row):
    print("neourn : {} - {} ".format(idx, arr))

neourn : 0 - [0] 
neourn : 1 - [9] 
neourn : 2 - [17] 
neourn : 3 - [25] 
neourn : 4 - [1, 18, 26] 
neourn : 5 - [10, 19, 27] 
neourn : 6 - [28] 
neourn : 7 - [2, 11] 
neourn : 8 - [12, 20] 
neourn : 9 - [3, 21, 29] 
neourn : 10 - [4, 30] 
neourn : 11 - [5, 13, 31] 
neourn : 12 - [6, 14, 22] 
neourn : 13 - [7, 15, 23] 
neourn : 14 - [8, 16, 24, 32] 


### Construct every layer bias(only VN update need bias(channel LLR))

In [13]:
# 神經網路裡面的bias=[LLR1 ... LLR_VN]*matrix
dot_bias_matrix=torch.zeros(parity_check_matrix_colsize ,hidden_layer_node)
idx=0
for i in range(0,len(VN2CN)):
    for j in range(0,len(VN2CN[i])):
        dot_bias_matrix[i][idx]=1;
        idx = idx + 1


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义模型参数
input_size = parity_check_matrix_colsize
hidden_layer_node = hidden_layer_node
output_size = parity_check_matrix_colsize
# VN update Layer
class VnUpdateLayer(nn.Module): # Apply Add
    def __init__(self, in_features, out_features,layer_mask):
        super(VnUpdateLayer, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias = nn.Parameter(torch.Tensor(out_features))
        self.layer_mask = torch.tensor(layer_mask, dtype=torch.float32)  # Convert to tensor
        self.reset_parameters()
        
    def reset_parameters(self):
        # nn.init.uniform_(self.weight,-1,1) # 初始化權重 range(-1,1)
        nn.init.ones_(self.weight) # 初始化權重 range(-1,1)
        self.weight.requires_grad=True
        nn.init.zeros_(self.bias)
        self.bias.requires_grad = False
        
        if self.layer_mask is not None:
            self.weight.data *= self.layer_mask

    def forward(self, x):
        return torch.matmul(x, self.weight.t())
    
    def apply_sparse_mask(self):
         # 将稀疏掩码为零的权重的梯度设为零
        if self.weight.grad is not None:
            self.weight.grad.data *= self.layer_mask

# CN update Layer (Dont need update paramter or weight)
class CnUpdateLayer(nn.Module): # apply dot
    def __init__(self, in_features, out_features,layer_mask,first=False):
        super(CnUpdateLayer, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias = nn.Parameter(torch.Tensor(out_features))
        self.out_features = out_features
        self.layer_mask = torch.tensor(layer_mask, dtype=torch.float32)  # Convert to tensor
        self.first = first
        self.reset_parameters()
        
    def reset_parameters(self):
       
        nn.init.ones_(self.weight) # 初始化權重 
        self.weight.requires_grad=False
        nn.init.zeros_(self.bias)
        self.bias.requires_grad = False
        
        if self.layer_mask is not None:
            self.weight.data *= self.layer_mask

    def forward(self, x):
        # 应用掩码并处理每个输出神经元
        batch_size = x.size(0)
        output = torch.zeros(batch_size, self.out_features, device=x.device)
        for i in range(self.out_features):
            # 应用每个输出神经元的掩码
            mask = self.layer_mask[i]
            x_masked = x * mask  # 掩码应用于输入数据
            # 将零值替换为一，以避免影响乘积
            x_nonzero = torch.where(x_masked != 0, x_masked, torch.ones_like(x_masked))
            # 计算掩码后数据的乘积
            product = torch.prod(x_nonzero, dim=1)
            # 检查全零行，将这些行的乘积结果置为1，避免影响后续的双曲正切反函数
            non_zero_prod = torch.where((x_masked != 0).any(dim=1), product, torch.zeros_like(product))
            # 计算双曲正切反函数并乘以2
            output[:, i] = non_zero_prod
        
        return output
    def apply_sparse_mask(self):
        pass
         # 将稀疏掩码为零的权重的梯度设为零
        # if self.weight.grad is not None:
        #     self.weight.grad.data *= self.layer_mask
            
class SparseBPNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,first_layer_mask,CN2VN_mask_VNUpdate,VN2CN_mask_CNUpdate,CN2VN_mask_output,dot_bias_matrix):
        super(SparseBPNeuralNetwork, self).__init__()
        self.fc0 = CnUpdateLayer(input_size, hidden_size,first_layer_mask,first=True) # CN update
        self.fc1 = VnUpdateLayer(hidden_size, hidden_size,CN2VN_mask_VNUpdate) # VN update (i=odd)
        self.fc2 = CnUpdateLayer(hidden_size, hidden_size,VN2CN_mask_CNUpdate,first=False) # CN update (i=even)
        self.fc3 = VnUpdateLayer(hidden_size, hidden_size,CN2VN_mask_VNUpdate) # VN update (i=odd)
        self.fc4 = CnUpdateLayer(hidden_size, hidden_size,VN2CN_mask_CNUpdate,first=False) # CN update (i=even)
        self.fc5 = VnUpdateLayer(hidden_size, output_size,CN2VN_mask_output) # output 
        self.bias_matrix = dot_bias_matrix
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        # first layer of input to CN update
        Channel_LLR = x.clone()
        x = 2*torch.atanh(self.fc0(self.tanh(0.5*x)))  # input to CNUpdate (i=1)
        x = self.tanh(0.5*(self.fc1(x)+torch.matmul(Channel_LLR,self.bias_matrix))) # CN->VN VNUpdate (i=2)
        x = 2*torch.atanh(self.fc2(x)) # VN->CN CNUpdate (i=3)
        x = self.tanh(0.5*(self.fc3(x)+torch.matmul(Channel_LLR,self.bias_matrix))) # CN->VN VNUpdate (i=4)
        x = 2*torch.atanh(self.fc4(x)) # VN->CN CNUpdate (i=5)
        x = self.sigmoid(self.fc5(x)+Channel_LLR) #(i=6)
        return x

    def apply_sparse_masks(self):
        self.fc1.apply_sparse_mask()



# create model
model = SparseBPNeuralNetwork(input_size,hidden_layer_node,output_size,first_layer_mask,CN2VN_mask_VNUpdate,VN2CN_mask_CNUpdate,CN2VN_mask_output,dot_bias_matrix)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# 打印初始的权重
print("Initial weights of fc0:", model.fc0.weight)
print("Initial weights of fc1:", model.fc1.weight)
print("Initial weights of fc2:", model.fc2.weight)
print("Initial weights of fc3:", model.fc3.weight)
print("Initial weights of fc4:", model.fc4.weight)
print("Initial weights of fc5:", model.fc5.weight)




Initial weights of fc0: Parameter containing:
tensor([[0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1.],
        [0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.],
        [0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1.],
        [0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1.],
        [0., 1., 0., 0., 0., 1., 0., 1., 1.,

In [18]:
# 生成一些训练数据
inputs = torch.tensor([[3.50427,3.56523,1.33009,3.96473,4.633,0.843421,2.99427,2.19127,0.299632,3.46877,-0.119053,0.38125,0.028388,3.19273,0.329816]])
targets = torch.zeros(1, output_size)
bias_input_hidden = torch.matmul(inputs,dot_bias_matrix)
bias_input_output = inputs.clone()
# 定义损失函数和优化器  
outputs=model(inputs)
print(outputs)
# train model
# for epoch in range(100):
#     optimizer.zero_grad()       # clear grad
#     outputs = model(inputs)     # forward
#     loss = criterion(outputs, targets)  # count loss
#     loss.backward()             # backward
#     model.apply_sparse_masks()  # 应用稀疏掩码，防止零权重更新
#     optimizer.step()            # update parameter

# print weights of fcs
# print("Trained weights of fc1:", model.fc1.weight.data)
# print("Trained weights of fc2:", model.fc2.weight.data)

tensor([[0.9708, 0.9725, 0.7909, 0.9814, 0.9904, 0.6987, 0.9523, 0.8995, 0.5746,
         0.9698, 0.4754, 0.5927, 0.5096, 0.9606, 0.5802]],
       grad_fn=<SigmoidBackward0>)


In [31]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # 定义一个简单的线性模型
# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
#         self.fc = nn.Linear(3, 1)
#         self.fc.weight.data.view(-1)[0]=0
#     def forward(self, x):
#         return self.fc(x)

# # 创建模型实例
# model = SimpleModel()

# # 打印初始的偏置
# print("Initial bias:", model.fc.bias.data)
# print("Initial weight:", model.fc.weight.data)
# # 定义损失函数和优化器
# criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01)

# # 生成一些训练数据
# inputs = torch.randn(10, 3)
# targets = torch.randn(10, 1)
# print(inputs)
# print(targets)
# # 训练模型
# for epoch in range(100):
#     optimizer.zero_grad()       # 清零梯度
#     outputs = model(inputs)     # 前向传播
#     loss = criterion(outputs, targets)  # 计算损失
#     loss.backward()             # 反向传播
#     optimizer.step()            # 更新参数

# # 打印训练后的偏置
# print("Trained bias:", model.fc.bias.data)
# print("Trained weight:", model.fc.weight.data)

Initial bias: tensor([-0.4180])
Initial weight: tensor([[ 0.0000, -0.1029, -0.3687]])
tensor([[ 1.2555, -1.7822,  0.7545],
        [ 1.1652,  0.3808, -0.7641],
        [ 0.5763,  0.0506, -1.3033],
        [-0.3080,  0.3904, -1.0683],
        [ 0.8905,  1.1731,  1.9236],
        [ 2.9778,  1.4531,  0.3056],
        [ 0.2717,  1.1332,  1.2300],
        [-0.4211,  0.0416,  1.1718],
        [-1.0794, -0.9284, -0.3353],
        [-0.9748, -1.3899, -1.3581]])
tensor([[ 2.7021],
        [-0.6055],
        [-1.7897],
        [ 0.2842],
        [-0.5957],
        [ 0.0611],
        [ 0.6374],
        [ 0.9967],
        [ 1.4977],
        [-0.4564]])
Trained bias: tensor([0.1559])
Trained weight: tensor([[ 0.0773, -0.6221,  0.4980]])


In [224]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class CnUpdateLayer(nn.Module):
#     def __init__(self, in_features, out_features, layer_mask):
#         super(CnUpdateLayer, self).__init__()
#         self.in_features = in_features
#         self.out_features = out_features
#         self.layer_mask = torch.tensor(layer_mask, dtype=torch.float32)  # Convert to tensor
#         self.reset_parameters()
    
#     def reset_parameters(self):
#         # 这里可以初始化其他参数，如果有的话
#         pass
    
#     def forward(self, x):
#         # 应用掩码并处理每个输出神经元
#         batch_size = x.size(0)
#         output = torch.zeros(batch_size, self.out_features, device=x.device)

#         for i in range(self.out_features):
#             # 应用每个输出神经元的掩码
#             mask = self.layer_mask[i]
#             x_masked = x * mask  # 掩码应用于输入数据
#             print(x_masked)
#             # 将零值替换为一，以避免影响乘积
#             x_nonzero = torch.where(x_masked != 0, x_masked, torch.ones_like(x_masked))
#             print(x_nonzero)
#             # 计算掩码后数据的乘积
#             product = torch.prod(x_nonzero, dim=1)
            
#             # 检查全零行，将这些行的乘积结果置为1，避免影响后续的双曲正切反函数
#             non_zero_prod = torch.where((x_masked != 0).any(dim=1), product, torch.zeros_like(product))
#             # print(non_zero_prod)
#             # 计算双曲正切反函数并乘以2
#             output[:, i] = 2 * torch.atanh(non_zero_prod)
        
#         return output

# # 定义掩码
# layer_mask = [
#     [0, 1, 1, 1, 1,1,1],
#     [0, 0, 1, 1, 1,1,1],
#     [0, 0, 0, 1, 1,1,1],
#     [0, 0, 0, 0, 1,1,1],
#     [1, 0, 0, 0, 1,1,1]
# ]

# # 创建层实例
# in_features = 7
# out_features = 5
# layer = CnUpdateLayer(in_features, out_features, layer_mask)

# # 随机生成一些输入数据，假设有一个batch size为3
# x_input = torch.ones(3, in_features)
# x_input = torch.tensor([[0.5,0.5,0.5,0.5,0.5,0.5,0.5],
#                         [0.5,0.5,0.5,0.5,0.5,0.5,0.5],
#                         [0,0,0,0,0,0,0]])
# # 通过层传递数据
# output = layer(x_input)

# print("Input:\n", x_input)
# print("Output:\n", output)


tensor([[0.0000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.0000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
tensor([[1.0000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])
tensor([[0.0000, 0.0000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.0000, 0.0000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
tensor([[1.0000, 1.0000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])
tensor([[0.0000, 0.0000, 0.0000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.0000, 0.0000, 0.0000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
tenso

In [ ]:
# x_mask=torch.tensor([0,1,1,1,1])
# non_zero_prod = torch.where((x_masked != 0).any(dim=1), product, torch.zeros_like(product))
# print()